# Setup

In [1]:
### Libraries
import pandas as pd
from IPython.display import display
### Python OBDC bridge
import pyodbc 
### IRIS Python Native API
import irisnative
### others...
import time

### SQL Connection parameters
dsn = 'IRIS IntegeratedML monitor'
server = 'irisimlsvr'
port = '51773'
database = 'USER' 
username = 'SUPERUSER' 
password = 'SYS' 
cnxn = pyodbc.connect('DRIVER={InterSystems ODBC35};SERVER='+server+';PORT='+port+';DATABASE='+database+';UID='+username+';PWD='+ password)

### charset stuffs...
cnxn.setdecoding(pyodbc.SQL_CHAR, encoding='utf8')
cnxn.setdecoding(pyodbc.SQL_WCHAR, encoding='utf8')
cnxn.setencoding(encoding='utf8')

### Get a SQL cursor
cursor = cnxn.cursor()

### Native API connection's parameters
ip = "irisimlsvr"
port = 51773
namespace = "USER"
username = "SUPERUSER"
password = "SYS"

### Create database connection and IRIS instance
try:
    nativeConn = irisnative.createConnection(ip, port, namespace, username, password)
    dbnative = irisnative.createIris(nativeConn)
except:
    print('Seems like you can\'t connect to IRIS... try to exit from any IRIS terminal or other license consumption session')

### Clean previous runs
df1 = pd.read_sql(
    "SELECT COUNT(*) AS ModelExists FROM INFORMATION_SCHEMA.ML_TRAINED_MODELS WHERE MODEL_NAME = 'AppointmentsPredection'", 
    cnxn
)
if (df1['ModelExists'][0] > 0):
    print('Deleting previous model...')
    cursor.execute("DROP MODEL AppointmentsPredection")
    print('Model deleted')
else:
    print('No previous runs found')

Deleting previous model...
Model deleted


# ML model setup

Let's create a model to predict appointments, using first 500 records to train it.

In [2]:
### Display dataset table
df1 = pd.read_sql(
    "SELECT * FROM MedicalAppointments", 
    cnxn
)
display(df1)

,ID,Canal,CreacionDiaD,CreacionHoraD,CreacionMesD,Edad,Especialidad,Latencia,ReservaDiaD,ReservaHoraD,ReservaMesD,Sexo,Show,Tipo
0,30662,3,1,12,1,46,22,45,4,10,3,2,True,1
1,30663,3,4,11,1,36,45,35,4,16,3,2,True,1
2,30664,3,3,13,1,14,21,36,4,15,3,1,True,1
3,30665,2,5,9,1,51,160,34,4,15,3,2,True,2
4,30666,2,4,14,12,47,74,70,4,9,3,1,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61209,61210,1,2,10,4,2,50,11,6,10,4,1,False,1
61210,61211,3,6,22,3,21,18,35,6,10,4,2,False,1
61211,61212,3,1,1,2,2,50,68,6,10,4,1,False,1
61212,61213,1,2,10,3,33,71,53,6,10,4,2,False,1


In [3]:
### Create a model for predicting patient's appointments misses
### Note: seed parameter it's to force reproducibility
cursor.execute(
    "CREATE MODEL AppointmentsPredection PREDICTING (Show) FROM MedicalAppointments USING {\"seed\": 3}"
)
print('Training model (this could take a while)...')
start = time.time()
cursor.execute(
    "TRAIN MODEL AppointmentsPredection FROM MedicalAppointments WHERE ID <= 500 USING {\"seed\": 3}"
)
cnxn.commit()
print('Model trained in %ss' % (time.time() - start))

Training model (this could take a while)...
Model trained in 23.2741060256958s


In [4]:
### Display model information
cursor.execute("SELECT * FROM INFORMATION_SCHEMA.ML_TRAINED_MODELS")
df1 = pd.read_sql("SELECT * FROM INFORMATION_SCHEMA.ML_TRAINED_MODELS", cnxn)
display(df1)

,MODEL_NAME,TRAINED_MODEL_NAME,PROVIDER,TRAINED_TIMESTAMP,MODEL_TYPE,MODEL_INFO
0,AppointmentsPredection,AppointmentsPredection2,AutoML,2020-09-25 19:10:58.873,classification,"ModelType:Logistic Regression, Package:sklearn..."


In [5]:
### Display prediction and real patient's appointments presence
df1 = pd.read_sql(
    "SELECT PREDICT(AppointmentsPredection) As Predicted, Show FROM MedicalAppointments WHERE ID <= 500" , 
    cnxn
)
display(df1)

,Predicted,Show
0,0,False
1,0,False
2,0,False
3,0,False
4,0,False
...,...,...
495,1,True
496,0,True
497,1,True
498,1,True


# Model monitoring

Now let's validate our model using 100 new records regargind train model, i.e., the 600 first records. Note that again the parameter seed was used in order to guarantee reproducibility.

In [6]:
### Model validation
cursor.execute(
    "VALIDATE MODEL AppointmentsPredection FROM MedicalAppointments WHERE ID < 600 USING {\"seed\": 3}"
)
df5 = pd.read_sql(
    "SELECT * FROM INFORMATION_SCHEMA.ML_VALIDATION_METRICS WHERE MODEL_NAME = 'AppointmentsPredection'" , 
    cnxn
)
df6 = df5.pivot(index='VALIDATION_RUN_NAME', columns='METRIC_NAME', values='METRIC_VALUE')
display(df6)

METRIC_NAME,Accuracy,F-Measure,Precision,Recall
VALIDATION_RUN_NAME,,,,
AppointmentsPredection21,0.9,0.94,0.98,0.91


As we can see, model's accuracy is about 90%.

MyMetric.IntegratedMLModelsValidation application monitor class let you to access such metrics. Thus it's possible to IRIS' ^%SYSMONMGR monitor utility being aware about your ML models perfomance.

Results can be seen by System Monitor default CSP interface: http://localhost:8092/csp/user/MyMetric.Sample.IntegratedMLModelsValidation.cls

With such functionality, someone could setup an alert into ^%SYSMONMGR monitor utility, checking model's performance metrics against thresholds. This way, IntegeratedML applications performance issues could be quickly notified.

# Simulating a performance issue

Let's re-validate the model, but this turn, using first 800 records and see how this affects to model's performance metrics.

In [7]:
### Model validation
cursor.execute(
    "VALIDATE MODEL AppointmentsPredection FROM MedicalAppointments WHERE ID < 800 USING {\"seed\": 3}"
)

Now model's accuracy had descreased to about 87%.  

Let's say that for you business rule, performances less than 90% are unacceptable. If you had previously setup an alert to notify your team when model's performance is less than 90%, such team could be quickly notified and take care about this issue.

Please refer to the System Monitor [documentation](https://irisdocs.intersystems.com/irislatest/csp/docbook/Doc.View.cls?KEY=GCM_healthmon) to see how setup such alert.

# Responding to notification of performance issue

An approach to deal with such performance issue could be simply retrain the model. However, there isn't a rule and each case demands properly analysis.

In [8]:
### Retrain model using first 600 records
print('Training model (this could take a while)...')
start = time.time()
cursor.execute(
    "TRAIN MODEL AppointmentsPredection FROM MedicalAppointments WHERE ID <= 600 USING {\"seed\": 3}"
)
cnxn.commit()
print('Model trained in %ss' % (time.time() - start))

### Model validation
cursor.execute(
    "VALIDATE MODEL AppointmentsPredection FROM MedicalAppointments WHERE ID <= 800 USING {\"seed\": 3}"
)

Training model (this could take a while)...
Model trained in 13.664696455001831s


# Teardown

In [9]:
### Close the database and native API connections
nativeConn.close()
cnxn.close()